In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
from modules.process_data import *
from modules.utils import *

/home/client/projects/sc_alz/sc_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-06-06 12:41:28.564850: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-06 12:41:29.110628: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
!python "modules/train_script.py" 256 15 0.0001 5 0 kan -s_layers 256 32

2024-06-06 12:41:30.734759: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-06 12:41:31.270805: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/client/projects/sc_alz/sc_env/lib/python3.10/site-packages/anndata/_core/anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
Number of pairs: 5
Batch_size: 256
Learning rate: 0.0001
Epochs: 15
Split: 0
Tag: kan
Hidden layers: [256, 32]
100%|█████████████████████████████████████████| 759/759 [01:28<00:00,  8.59it/s]
Epoch 0, Train Loss: 8.4922441217425, Validation Accuracy: 0.49729173103956026
Model saved as best model
100%|██████████████████████████████

In [ ]:
!python "modules/eval_script.py" kan kan_0_5_32.pth 0 -s_layers 256 32

In [ ]:
X, Y = pickle.load(open('embed_kan_0_5_32.pth.pkl', 'rb'))

In [ ]:
#PCA dim. reduction
#pca = PCA(n_components=10, random_state=42)
#pca_result = pca.fit_transform(X)

In [ ]:
#UMAP dim. reduction
reducer = UMAP(n_neighbors=100, n_components=2, random_state=42)
umap_result = reducer.fit_transform(X)

In [ ]:
mapping2 = {'LD_5xFAD': 1,
            'LD_NC': 2,
            'run_5xFAD': 3,
            'run_NC': 4}

mapping2 = {'Multiplet': 1,
            'SampleTag17_flex': 2,
            'SampleTag18_flex': 3,
            'SampleTag19_flex': 4,
            'SampleTag20_flex': 5,
            'Undetermined': 6}

In [ ]:
mapping = {y:x for x,y in mapping1.items()}

In [ ]:
visualize_umap(umap_result, Y, mapping)

In [ ]:
#KMEANS clustering
kmeans = KMeans(n_clusters=2, random_state=42)
kmeans_result = kmeans.fit_predict(umap_result)

In [ ]:
#DBSCAN clustering
dbscan = DBSCAN(eps=0.5, min_samples=100)
dbscan_result = dbscan.fit_predict(umap_result)

In [ ]:
get_clustering(umap_result, dbscan_result)

In [ ]:
cluster_composition = get_cluster_composition(dbscan_result, Y)

In [ ]:
plot_cluster_composition(cluster_composition, mapping)

In [ ]:
get_pie_chart(cluster_composition[2], mapping, min_pct=5)

In [ ]:
from torch import nn
from modules.kan_model import DeepKAN
import torch
from matplotlib import pyplot as plt

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
input_dim = 29535
shared_layers = [1024,256,32]
num_knots = 5
spline_order = 3
noise_scale = 0.1
base_scale = 1.0
spline_scale = 1.0
activation = nn.SiLU
grid_epsilon = 0.02
grid_range = [-1, 1]

In [ ]:
base_net = DeepKAN(input_dim, shared_layers, num_knots, spline_order,
                noise_scale, base_scale, spline_scale, activation, grid_epsilon, grid_range)

In [ ]:
model_path = "kan_0_5_32.pth"
checkpoint = torch.load(model_path, map_location=device)
base_net.load_state_dict(checkpoint)
base_net.to(device)

In [ ]:
# Prune the network
pruning_threshold = 0.01
base_net.prune_network(pruning_threshold)
mask = [(param != 0).float() for param in base_net.parameters()]
def apply_pruning_mask(model, mask):
    with torch.no_grad():
        for param, mask in zip(model.parameters(), mask):
            param.mul_(mask)
apply_pruning_mask(base_net, mask)

In [ ]:
def print_network_parameters(model):
    for i, layer in enumerate(model.layers):
        print(f"Layer {i + 1}:")
        print("Base Weights:")
        print(layer.base_weights.data)
        print("Spline Weights:")
        print(layer.spline_weights.data)
        if layer.standalone_spline_scaling:
            print("Spline Scales:")
            print(layer.spline_scales.data)
        print("-" * 50)

In [ ]:
print_network_parameters(base_net)

In [ ]:
plt.plot(base_net.layers[0].spline_weights[0][0].cpu().detach().numpy())